In [1]:
import requests
from pprint import pprint
url = "https://ocean.defichain.com/v0/mainnet/loans/vaults"


In [ ]:
def fetch_and_analyze_vaults(url):
    """Fetch and analyze active vaults with loans and collateral"""
    # Fetch the list of vaults with pagination support
    params = {"size": 100}  # Fetch 100 vaults per request
    all_vaults = []
    
    while True:
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Failed to fetch vault data: {response.status_code}, {response.text}")
            return None
            
        data = response.json()
        all_vaults.extend(data["data"])
        
        # Check if there are more vaults to fetch
        if "page" not in data or not data["page"].get("next"):
            break
            
        params["next"] = data["page"]["next"]

    # Filter active vaults with loans and collateral
    active_vaults = [
        vault for vault in all_vaults
        if float(vault.get("loanValue", 0)) > 0 and float(vault.get("collateralValue", 0)) > 0
    ]

    # Print summary statistics
    print(f"Total vaults: {len(all_vaults)}")
    print(f"Active vaults with loans and collateral: {len(active_vaults)}")
    print("\nDetails of active vaults:")

    # Print details of filtered vaults with improved formatting
    for vault in active_vaults:
        print("\n" + "="*50)
        print(f"Owner Address: {vault['ownerAddress']}")
        print(f"Vault ID: {vault['vaultId']}")
        print(f"Collateral Ratio: {vault['collateralRatio']}%")
        print(f"Collateral Value: ${float(vault['collateralValue']):,.2f}")
        print(f"Loan Value: ${float(vault['loanValue']):,.2f}")
        print(f"Loan Scheme: {vault['loanScheme']['id']} (Min CR: {vault['loanScheme']['minColRatio']}%, Interest: {vault['loanScheme']['interestRate']}%)")
        
        print("\nCollateral Details:")
        for collateral in vault.get("collateralAmounts", []):
            print(f"  - {collateral['symbol']}: {float(collateral['amount']):,.8f}")
            
        print("\nLoan Details:")
        for loan in vault.get("loanAmounts", []):
            print(f"  - {loan['symbol']}: {float(loan['amount']):,.8f}")

    return active_vaults

# Execute the function
active_vaults = fetch_and_analyze_vaults(url)


In [5]:
def create_asset_to_perp_mapping(assets):
    return {asset['asset']: asset['perp'] for asset in assets}

# Example usage
assets = [
    { "asset": "BTC", "perp": "XBTUSD" },
    { "asset": "ETH", "perp": "ETHUSD" },
    { "asset": "XRP", "perp": "XRPUSD" },
    { "asset": "SOL", "perp": "SOLUSD" },
    { "asset": "SUI", "perp": "SUIUSD" },
    { "asset": "DOGE", "perp": "DOGEUSDT" },
    { "asset": "LTC", "perp": "LTCUSD" },
    { "asset": "ADA", "perp": "ADAUSD" },
    { "asset": "LINK", "perp": "LINKUSD" },
    { "asset": "PEPE", "perp": "PEPEUSD" },
    { "asset": "DOT", "perp": "DOTUSD" },
    { "asset": "BNB", "perp": "BNBUSD" },
    { "asset": "BCH", "perp": "BCHUSD" },
    { "asset": "AVAX", "perp": "AVAXUSD" },
    { "asset": "NEAR", "perp": "NEARUSD" },
    { "asset": "WLD", "perp": "WLDUSD" },
    { "asset": "APT", "perp": "APTUSD" },
    { "asset": "FIL", "perp": "FILUSD" },
    { "asset": "ARB", "perp": "ARBUSD" }
]

asset_to_perp = create_asset_to_perp_mapping(assets)

In [6]:
# Function to filter and sort vaults by any asset collateral
def get_vaults_by_asset(asset_symbol, vaults):
    # Filter vaults that contain the specified asset as collateral
    filtered_vaults = [
        vault for vault in vaults 
        if any(c['symbol'] == asset_symbol for c in vault.get('collateralAmounts', []))
    ]
    
    # Sort vaults by collateral value in descending order
    asset_vaults = sorted(
        filtered_vaults,
        key=lambda x: float(x['collateralValue']),
        reverse=True
    )
    return asset_vaults


In [7]:

# Create dictionaries to store vaults by asset and by owner
vaults_by_asset = {}
vaults_by_owner = {}

# Process vaults for all assets in the assets list
for asset in assets:
    asset_symbol = asset['asset']
    asset_vaults = get_vaults_by_asset(asset_symbol, active_vaults)
    
    # Store the results in the dictionary
    vaults_by_asset[asset_symbol] = []
    
    for vault in asset_vaults:
        # Extract the specific collateral amount for this asset
        collateral_amount = next(
            (c['amount'] for c in vault['collateralAmounts'] 
             if c['symbol'] == asset_symbol),
            '0.00000000'  # Default value if not found
        )
        
        # Create vault data structure
        vault_data = {
            'ownerAddress': vault['ownerAddress'],
            'vaultId': vault['vaultId'],
            'collateralValue': float(vault['collateralValue']),
            'collateralAmount': collateral_amount,
            'symbols': [c['symbol'] for c in vault['collateralAmounts']]
        }
        
        # Add to the asset's vault list
        vaults_by_asset[asset_symbol].append(vault_data)
        
        # Update owner's total collateral
        owner = vault['ownerAddress']
        if owner not in vaults_by_owner:
            vaults_by_owner[owner] = {
                'totalCollateralValue': float(vault['collateralValue']),  # Initialize with this vault's value
                'vaults': [vault_data]
            }
        else:
            # Only add the vault data, don't add to total collateral value
            # since it's already the total value for the vault
            vaults_by_owner[owner]['vaults'].append(vault_data)


In [8]:

# Sort owners by total collateral value in descending order
vaults_by_owner_collateral = sorted(
    vaults_by_owner.items(),
    key=lambda x: x[1]['totalCollateralValue'],
    reverse=True
)


In [ ]:
for owner, vault_info in vaults_by_owner_collateral:
    print(f"Owner: {owner}, Total Collateral Value: ${vault_info['totalCollateralValue']:,.2f}")
    for vault in vault_info['vaults']:
        # Convert collateralAmount to float before formatting
        collateral_amount = float(vault['collateralAmount'])
        print(f"  - Vault ID: {vault['vaultId']}, Collateral Value: ${vault['collateralValue']:,.2f}, Collateral Amount: {collateral_amount:,.8f}, Symbols: {', '.join(vault['symbols'])}")


In [ ]:
# Get the top owners by total collateral value
top_owner_number = 10
top_owners = vaults_by_owner_collateral[:top_owner_number]

# Create a set to store unique vault IDs from top owners
top_vaults = set()

# Iterate through top owners and collect their vaults
for owner, vault_info in top_owners:
    for vault in vault_info['vaults']:
        top_vaults.add(vault['vaultId'])

# Create a dictionary to store vaults by collateral asset
vaults_by_asset = {}

# Iterate through top owners and organize vaults by collateral asset
for owner, vault_info in top_owners:
    for vault in vault_info['vaults']:
        for symbol in vault['symbols']:
            if symbol not in vaults_by_asset:
                vaults_by_asset[symbol] = set()
            vaults_by_asset[symbol].add(vault['vaultId'])

# Print number of unique vaults and breakdown by collateral asset
print(f"Number of unique vaults in top {top_owner_number} owners: {len(top_vaults)}")
print("\nVaults by collateral asset:")
for asset, vault_ids in vaults_by_asset.items():
    print(f"  - {asset}: {len(vault_ids)} vaults")


In [ ]:
def get_current_vault_details(vault_id, url):
    vault_url = f"{url}/{vault_id}"
    response = requests.get(vault_url)
    return response.json()['data']

def get_weighted_collaterals(vault_ids):
    weighted_collateral_for_asset = {}
    for asset, vault_ids in vaults_by_asset.items():
        total_collateral_for_asset = 0
        total_collateral_for_asset_ratio = 0
        print(f"Asset: {asset}")
        for vault_id in vault_ids:
            vault_details = get_current_vault_details(vault_id)
            collateral_ratio = float(vault_details['collateralRatio'])
            total_collateral = float(vault_details['collateralValue'])
            total_collateral_for_asset += total_collateral
            total_collateral_for_asset_ratio += collateral_ratio*total_collateral
            print(f"  - Vault ID: {vault_id}, Collateral Ratio: {float(vault_details['collateralRatio'])}%, Total Collateral: ${float(vault_details['collateralValue']):,.2f}")
        print(f"  - Total Collateral for Asset: ${total_collateral_for_asset:,.2f}")
        print(f"  - Total Collateral for Asset Ratio: ${total_collateral_for_asset_ratio:,.2f}")
        print(f"  - Weighted Collateral for Asset: ${total_collateral_for_asset_ratio/total_collateral_for_asset:.2f}")
        weighted_collateral_for_asset[asset] = total_collateral_for_asset_ratio/total_collateral_for_asset
    return weighted_collateral_for_asset

print(len(top_vaults))
weighted_collaterals = get_weighted_collaterals(top_vaults)


In [ ]:
def decide_signal(self, src, asset, collateral_ratio):
    """Makes trading decisions based on the collateral ratio."""
    if collateral_ratio < 150:  # Risk of liquidation
        size = -1.0  # Go fully short
    elif collateral_ratio > 200:  # Vault is safe
        size = 1.0  # Go fully long
    else:
        # Smooth transition between -1 and 1 based on collateral ratio
        # Normalize ratio to 0-1 range between 150-200
        normalized_ratio = (collateral_ratio - 150) / 50
        # Map to -1 to 1 range
        size = 2 * normalized_ratio - 1
